In [20]:
import os
import wfdb
import numpy as np
import pandas as pd

In [22]:
records = np.loadtxt('D:/phd/mit-bih-atrial-fibrillation-database-1.0.0/files/RECORDS', dtype=str)[2:]
pairs_dict = dict()
len_persantage = dict()
for record_number in records:
    filepath = 'D:/phd/mit-bih-atrial-fibrillation-database-1.0.0/files/' + record_number
    event_length = 0
    event_count = 0
    record = wfdb.rdrecord(filepath)
    annotation = wfdb.rdann(filepath, 'atr', shift_samps=True)
    for i, val in enumerate(annotation.aux_note):
        if val == '(AFL' or val == '(AFIB':
            start, end = [annotation.sample[i], annotation.sample[i+1] if i + 1 < len(annotation.aux_note) else len(record.p_signal) - 1]
            event_length += end - start
            event_count += 1
    len_persantage[record_number] = [event_length, event_count, len(record.p_signal)]


In [26]:
for key, val in len_persantage.items():
    print(key, 'record has', val[1], 'events and', val[0]/val[2], 'of AF events')

04015 record has 7 events and 0.006442596808954394 of AF events
04043 record has 83 events and 0.21542501651140156 of AF events
04048 record has 7 events and 0.009797561526696329 of AF events
04126 record has 7 events and 0.037421462214961065 of AF events
04746 record has 5 events and 0.5309992874026697 of AF events
04908 record has 12 events and 0.09059729995133482 of AF events
04936 record has 37 events and 0.813393136471079 of AF events
05091 record has 8 events and 0.002356350806451613 of AF events
05121 record has 20 events and 0.6302832139877642 of AF events
05261 record has 11 events and 0.01302152130839822 of AF events
06426 record has 27 events and 0.9567735852335929 of AF events
06453 record has 6 events and 0.01114954954954955 of AF events
06995 record has 8 events and 0.47172476797135704 of AF events
07162 record has 1 events and 0.9999809901626807 of AF events
07859 record has 1 events and 0.9999952203837598 of AF events
07879 record has 2 events and 0.6029564098998887 of 

In [27]:
def events_count(path, record, results):
    arr = np.loadtxt(path, dtype = int)
    events = 0
    right_events = 0
    for i in range(len(arr)):
        if results[record][0] <= i <= results[record][1] and arr[i] == 1 and arr[i-1] == 0:
                right_events += 1
        if arr[i] == 1 and arr[i-1] == 0:
            events += 1
    interval = results[record][1]- results[record][0]
    return events, right_events, 100* right_events/(events) if events > 0 else 0, interval

In [28]:
results = dict()
results['05261'] = [2162708-2158708, 2163556-2158708]
results['06426'] = [534235-524235,  556304-524235]
results['07879'] = [7334783-7331783, 7335468-7331783]
results['08434'] = [2237188-2232188, 2315413-2232188]
results['08378'] = [742007-722000, 760444-722000]
for key, value in results.items():
    results[key] = [value[0]/8, value[1]/8]

folder = "D:\\test\\CheckProjJulia\\out\\fft\\"
answers = []
for root, dirs, files in os.walk(folder, topdown=False):
    for file in files:
        penalty = file.split(' ')[1].split('.')[0]
        total_events, right_events, right_percent, *_ = events_count(folder + file, file[:5], results)
        answers.append([file[:5], penalty, total_events, right_events, right_percent])
        
fft_statictics = pd.DataFrame(answers, columns = ['record','penalty','total events', 'events overlap', 'correct %'])

In [29]:
results = dict()
results['05261'] = [2162708-2158708, 2163556-2158708]
results['06426'] = [534235-524235,  556304-524235]
results['07879'] = [7334783-7331783, 7335468-7331783]
results['08434'] = [2237188-2232188, 2315413-2232188]
results['08378'] = [742007-722000, 760444-722000]

folder = "D:\\test\\CheckProjJulia\\out\\normal\\"
answers = []
for root, dirs, files in os.walk(folder, topdown=False):
    for file in files:
        parts = file.split('traceback')
        total_events, right_events, right_percent, *_ = events_count(folder + file, file[:5], results)
        answers.append([parts[0], int(parts[1].split('.')[0][1:]), total_events, right_events, right_percent])
        
normal_statictics = pd.DataFrame(answers, columns = ['record','penalty','total events', 'events overlap', 'correct %'])

# Processed without FFT

In [16]:
pd.options.display.max_rows = 250
normal_statictics = normal_statictics.sort_values(by=['record', 'penalty']).reset_index(drop=True)
normal_statictics

,record,penalty,total events,events overlap,correct %
0,05261,5,34,6,17.647059
1,05261,5,14,1,7.142857
2,05261,10,24,4,16.666667
3,05261,10,6,0,0.000000
4,05261,20,9,0,0.000000
5,05261,20,8,0,0.000000
6,05261,50,4,0,0.000000
7,05261,50,1,0,0.000000
8,05261,100,6,0,0.000000
9,05261,100,1,0,0.000000


# Processed with FFT

In [15]:
pd.options.display.max_rows = 250
fft_statictics = fft_statictics.sort_values(by=['record', 'penalty']).reset_index(drop=True)
fft_statictics

,record,penalty,total events,events overlap,correct %
0,05261,10,39,5,12.820513
1,05261,10,9,2,22.222222
2,05261,100,6,0,0.000000
3,05261,100,0,0,0.000000
4,05261,1000,0,0,0.000000
5,05261,1000,0,0,0.000000
6,05261,20,16,2,12.500000
7,05261,20,6,3,50.000000
8,05261,200,4,0,0.000000
9,05261,200,0,0,0.000000
